# Exercise 05: Gaussian Process Model Predictive Control (GP-MPC)

## 1. Introduction

Model mismatch is a frequent source of control inaccuracy in real-world applications of MPC. Since MPC is a optimization-based control method, MPCs will try to exploit the given model dynamics as much as possible, which may cause it to become instable with the real dynamics.

One research direction is to find a good approximation of the difference $d(x,u)$ between the nominal model $f_{\text{nominal}}(x,u)$ and the real model, i.e.,

$$ \dot{x} = f_{\text{nominal}}(x,u) + d(x,u),$$

This approximation can be achieved via learning-based methods, and Gaussian Processes (GPs) are one such method. 

GPs have the inherent advantage that they also provide uncertainty estimates which can be used to ensure stochastic constraint satisfaction and to improve the robustness of the MPC controller.

In this exercise, we focus on GP-MPC which models the residual dynamics with GPs and uses the uncertainty estimates to tighten the future state and input constraints.

We then compare the performance between GP-MPC and nominal MPC when tracking a trajectory with a simulated drone under model mismatch.

### Imports

In [ ]:
# Commented out because it can cause issues when changing the gpmpc parameters
# As a workaround we (re-)import the required modules at the beginning of each cell
# %load_ext autoreload
# %autoreload 2

from pathlib import Path

import crazyflow  # noqa: F401, required for registering environments
import gymnasium
import matplotlib.pyplot as plt
import numpy as np
import torch
from exercise05.run_gpmpc import Evaluator
from exercise05.utils import remove_path, set_seed
from gymnasium.wrappers.vector.jax_to_numpy import JaxToNumpy

# select global seed and device
global_seed = 0
set_seed(global_seed)
if not torch.cuda.is_available():
    device = "cpu"
else:
    device = "cuda"
print(f"Using device: {device}")

### Create the Environment and the Trajectory to Follow
First, we create an environment with a figure-eight trajectory in the x–y plane as the reference trajectory, extract the trajectory, and the action space limits. Then we distort the (identified) drone dynamics to simulate model mismatch.

In [ ]:
env = JaxToNumpy(gymnasium.make_vec("DroneFigureEightTrajectory-v0", num_envs=1))
traj = env.unwrapped.trajectory.T
# Pad the trajectory with zeros to match the state dimension (12)
traj = np.concatenate([traj, np.zeros((9, traj.shape[1]))], axis=0)
env.unwrapped.render_trajectory = False

# Correct params of the dynamics model
correct_params = {
        "a": 20.9,
        "b": 3.6,
        "ra": -130,
        "rb": -16.3,
        "rc": 119.3,
        "pa": -100.0,
        "pb": -13.3,
        "pc": 84.47,
        "ya": -0.01,
        "yb": 0.0,
        "yc": 0.0,
    }

def disturb_params(params, scale=0.1):
    params = params.copy()
    for k, v in params.items():
        params[k] = v * (1 + scale * (2 * np.random.rand() - 1))
    return params

# Plot the trajectory
plt.figure(figsize=(5, 3))
plt.plot(traj[0, :], traj[2, :])
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.title("Trajectory")


## 2. Implement the GP‑MPC Controller

We augment the nominal dynamics with a learned residual model
$$
x_{k+1}=f_{\text{nom}}(x_k,u_k)+d(x_k,u_k),
$$
where the residual $d(\cdot)$ is represented by a small set of sparse variational Gaussian Processes (SVGPs). We then (i) construct an efficiently evaluable residual parameterization, (ii) embed it into the discrete MPC dynamics, and (iii) tighten constraints using the GP predictive variance to achieve probabilistic satisfaction.

### 2.1 Parameterize the Residual Dynamics with Sparse Variational GPs

**Why sparse & low–dimensional?** MPC requires hundreds–thousands of repeated dynamics + Jacobian evaluations per second. A naïve GP with $N$ data points costs $\mathcal{O}(N^3)$ to train and $\mathcal{O}(N)$ per query. We therefore:
1. Use an SVGP (variational inducing point approximation) with $M\ll N$.
2. Decompose the residual into a *small* set of independent scalar GPs over carefully chosen, low‑dimensional input groups (feature grouping).
3. Export only the minimal quantities required inside the solver (inducing inputs + a compressed weight vector $\alpha$).

**Grouping used (attitude + thrust residual):**
- GP 1: thrust command  → thrust residual (mapped to body accelerations).
- GP 2: (φ, dφ, φ_cmd)   → roll‑rate residual.
- GP 3: (θ, dθ, θ_cmd)   → pitch‑rate residual.

Each scalar GP provides a predictive mean; we ignore cross‑covariances for speed (diagonal residual covariance).

**Sparse parameterization inserted into CasADi:**
For an input $z\in\mathbb{R}^{d}$ and inducing set $Z\in\mathbb{R}^{M\times d}$,
$$
\mu(z)\approx k(z,Z)\,K_{ZZ}^{-1} m_u \;\; \Longrightarrow \;\; \mu(z)=k(z,Z)\,\alpha,
$$
where we precompute $\alpha=K_{ZZ}^{-1} m_u$. We pass the flattened vector
$$
\theta = [\alpha,\; Z]
$$
as an MPC parameter. Each GP contributes its own $\theta_i$; all are concatenated.

**Resulting residual vector build:**
- Convert thrust residual to (a_x,a_y,a_z) with current attitude.
- Insert angular rate residuals in the appropriate state derivative components.
- Unspecified components remain zero.

**Numerical stability:** GPs can be very sensitive to numerical issues. This can be counteracted by: Appropriate hyperparameter initialization, enforcing lower bounds on lengthscales, using a numerically robust Cholesky decomposition, and selecting the inducing points well.

<div class="alert alert-info">
  <h3>Task 1: Parameterize the Residual Dynamics</h3>
  <p> Task 1.1: Review the nominal model: <code>symbolic_attitude</code> and <code>SymbolicModel</code> (Crazyflow package). </p>
  <p> Task 1.2: Inspect <code>exercise05/gp.py</code>, then complete following methods of the <code>_SingleOutputSVGP</code> class: </p>
    <ul>
        <li><code>get_sparse_parameterization</code>: create a casadi expression of &mu;(z) as a function of &alpha; and the inducing points.</li>
        <li><code>get_sparse_parameters</code>: extract &alpha; and the inducing points from the trained GP model.</li>
    </ul>
</div>

### 2.2 Embedding the Residual GP in the MPC Dynamics

We create a modified Acados model:
$x_{k+1}= \Phi_{\Delta t}\big(f_{\text{nom}}(x_k,u_k)+d(x_k,u_k)\big),$
implemented by replacing the discrete dynamics expression with RK4 over the *augmented* continuous field.

Steps performed in `GPMPC` (see `exercise05/gpmpc.py`):
1. Collect state & input symbols; build grouped GP inputs (`gp_inputs` list).
2. Obtain symbolic residual expressions + parameter symbols from each GP (`get_sparse_parameterization`).
3. Assemble the residual dynamics vector from the individual GP means.
4. Register a single concatenated parameter vector (`gp_params`) with the OCP registry (enables fast in‑place updates after re‑training).
5. Replace `disc_dyn_expr` with RK4 discretization of nominal + residual dynamics.

After (re)training:
- Call `self.gpmpc_registry.update_params(..., which=["gp_params"])` to push the updated parameters into the solver without regeneration of code.
<div class="alert alert-info">
  <h3>Task 2: Augment MPC with Residual Dynamics</h3>
  <p> Task 2.1 Review and understand the <code>_add_residual_dynamics_to_acados_model</code> method in <code>exercise05/gpmpc.py</code> </p>
</div>

### 2.3 Probabilistic Constraint Tightening via GP Uncertainty

We want
$$
\Pr( A_x x_k \le b_x ) \ge p,\qquad \Pr( A_u u_k \le b_u ) \ge p
$$
under additive residual uncertainty. We linearize around the current planned trajectory and propagate a Gaussian covariance.

Linearized deviation dynamics (nominal + residual mean):
$$
\delta x_{k+1}=A_k \delta x_k + B_k \delta u_k + B_{d,k} w_k,\quad w_k\sim\mathcal{N}(0,\Sigma_{d,k})
$$
with:
- $A_k=\frac{\partial f_{\text{nom}}}{\partial x}$,
- $B_k=\frac{\partial f_{\text{nom}}}{\partial u}$,
- $B_{d,k}$ maps residual components to the affected state derivatives (sparse),
- $\Sigma_{d,k}=\operatorname{diag}(\sigma^2_{\text{GP},k})$ (diagonal, per GP output).

Covariance recursion:
$$
\Sigma_{x,k+1}=A_k \Sigma_{x,k} A_k^\top + B_k \Sigma_{u,k} B_k^\top + B_{d,k} \Sigma_{d,k} B_{d,k}^\top
+ A_k \Sigma_{x,u,k} B_k^\top + B_k \Sigma_{x,u,k}^\top A_k^\top
$$
In our simplified implementation we:
1. Start with $\Sigma_{x,0}=0$.
2. Obtain GP predictive variances along the *current* nominal trajectory (treating controls fixed).
3. Propagate forward with a precomputed stabilizing LQR gain $K$ used to approximate the closed‑loop input fluctuation: $\delta u_k \approx K \delta x_k$.
4. Extract per‑component standard deviations.

Constraint tightening (Bonferroni / union bound style):
For each scalar inequality $a_i^\top x \le b_i$, replace with
$$
a_i^\top x \le b_i - \Phi^{-1}(p_i)\,\sqrt{a_i^\top \Sigma_x a_i},
$$
where $p_i$ is chosen so overall probability ≥ desired `prob`. Code precomputes a conservative quantile (see `inverse_cdf` in `GPMPC.__init__`).

Margins for inputs (analogous) use $\Sigma_u = K\Sigma_x K^\top$.

<div class="alert alert-info">
  <h3>Task 3: Implement the Constraint Tightening</h3>
  <p> Task 3.1: Complete the <code>create_tightened_constraints</code> method in <code>exercise05/gpmpc.py</code>. </p>
    <p> Task 3.2: Complete the <code>setup_prior_dynamics</code> function in <code>exercise05/mpc.py</code> .</p>
    <p> Task 3.3 (Optional) Review and complete <code>update_tightening_parameters</code> in <code>exercise05/gpmpc.py</code> and ensure the covariance propagation and margin computation matches the above explanations. This is a bonus task and not required to complete the exercise. When not completing the function, set prob to 0.0 in the config to disable tightening.</p>
</div>

### 2.4 Training Loop & Evaluation

We iteratively:
1. Roll out current (GP-)MPC → collect (x,u,x') data.
2. Add residual samples to replay buffer (`add_data`).
3. (Re)fit SVGPs (`fit`), update parameters in solver.
4. Re-run evaluation episodes and log tracking MSE.

<div class="alert alert-info">
  <h3>Task 4: Run and Evaluate the GP-MPC</h3>
  <p> Task 4.1: Run and compare nominal MPC (epoch 0) vs successive GP-MPC epochs (plots + MSE). </p>
  <p> Task 4.2: Tune hyperparameters: inducing points, horizon, cost weights, constraint probability level. </p>
  <p> Task 4.3: Discuss: diminishing returns, effect of probability level, effect of inducing point count. </p>
</div>

In [ ]:
from exercise05.gpmpc import GPMPC  # noqa: F401, reimport for edits
from exercise05.mpc import MPC  # noqa: F401, reimport for edits

ctrl = None
output_dir = Path(Path.cwd() / "ctrl_outputs")
remove_path(output_dir)  # Clean up previous code to prevent interference
output_dir.mkdir(parents=True, exist_ok=True)

# Set distortion scale
prior_params = disturb_params(correct_params, scale=1.0)
prior_params["a"] = 5.0  # Make it really bad

# Parameters to play with
cfg = {
    "prob": 0.0, # Only change if you implemented the constraint tightening
    "replay_buffer": {"max_size": 2000, "data_selection": "rnd"},
    "mpc": {
        "horizon": 70,
        "cost_params": {"Q": [100, 0.1, 100, 0.5, 0.5, 0.5, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01], "R": [4, 0.5, 0.5, 0.5]},
    },
    "gp": {"n_inducing_points": 20},
    "gp_train": {"lr": 1e-3, "epochs": 200, "patience": 20},
}

ctrl = GPMPC(env=env.unwrapped, prior_params=prior_params, cfg=cfg, output_dir=output_dir, device=device)


In [ ]:
# Note: If the optimization fails, adjust the parameters and reroll the distortion (or set manually). You can ignore "adding jitter" warnings. Also, the GP-MPC training and evaluation can take a while, especially when using the CPU. Adjust the number of epochs, the replay buffer size, the number of inducing points, and the number of evaluation epochs to speed this up if necessary.
evaluator = None
eval_env = gymnasium.make_vec("DroneFigureEightTrajectory-v0", num_envs=1)
evaluator = Evaluator(eval_env, ctrl, seed=global_seed, verbosity="info")
evaluator.run(n_epochs=3, extra_train_runs=True)
evaluator.show_results()

In [ ]:
# Test dynamic quality of the learned model
run = evaluator.runs[list(evaluator.runs.keys())[-1]]
ctrl.evaluate_dynamics_quality(test_x=run["obs"][:-1, :], test_u=run["action"], test_x_next=run["obs"][1:, :], plot=True)

## Discussion: Performance of MPC and GP-MPC Controllers

#### Reasons for Weak Performance of MPC-based Controllers

- **Control Constraints & Cost Function Tuning:**  
  The MPC’s tracking performance is fundamentally limited by actuator constraints (e.g., thrust, rate limits) and the weights in the cost function. If the cost function is not carefully tuned to prioritize tracking the most important states (such as x and y positions), or if input penalties are too high, the controller may avoid aggressive maneuvers and tolerate tracking errors. Hard constraints can also prevent the controller from following aggressive or sharp trajectory segments.

- **Reference Trajectory Feasibility:**  
  The reference trajectory itself may be physically infeasible or require actions that exceed the system’s capabilities. Even with a perfect model, the controller cannot track a trajectory that is not dynamically feasible given the system’s physical and actuation limits.

- **Model Structure, Underactuation, and Real-World Effects:**  
  Even with a perfect model, real-world factors such as unmodeled dynamics, sensor noise, disturbances, and discretization errors can degrade tracking. Underactuated systems (where not all degrees of freedom are directly controlled) further limit tracking performance.

- **Prediction Horizon Limitations:**  
  The finite prediction horizon of MPC means the controller can only optimize over a limited future window. A short horizon can limit the ability to anticipate and plan for future trajectory changes, especially for aggressive or rapidly changing references.

#### Reasons for Limited Improvement with GP-MPC or Even Worse Performance

- **Marginal Gains After Initial Learning:**  
  GP-MPC often shows some improvement over nominal MPC after initial training, as the GP learns to correct systematic model errors. However, further improvements typically plateau. The GP can only correct for model errors it has seen in the training data, and its generalization is limited by the diversity and representativeness of that data.

- **Data Efficiency & Generalization:**  
  GPs require sufficient and representative data to accurately model the residual dynamics. If the training data does not cover the full range of operating conditions, or is too limited, the GP’s corrections will be incomplete or inaccurate.

- **Model Structure & Expressiveness:**  
  The GP’s input selection, kernel choice, and output structure may not be expressive enough to capture all relevant model errors. Over-regularization or poor hyperparameter choices can lead to underfitting.

- **Uncertainty Propagation & Constraint Tightening:**  
  GP-MPC propagates uncertainty through the dynamics and tightens constraints to ensure probabilistic safety. If the GP’s uncertainty estimates are inaccurate, or if the constraint tightening is too conservative, the controller may become overly cautious, reducing its ability to exploit the learned model and track the reference closely.

- **Computational and Practical Limitations:**  
  Sparse GP approximations (e.g., FITC) are used for tractability, but can reduce GP accuracy, especially for highly nonlinear or high-dimensional systems. Limited computational resources may restrict the number of inducing points or training iterations.

#### Key Takeaways

- **MPC performance is fundamentally limited by system constraints, cost function design, and model accuracy.** Even with a perfect model, perfect tracking is generally not possible due to these limitations.
- **GP-MPC can improve performance by learning and compensating for model errors,** but its effectiveness is limited by the quality and quantity of training data, the expressiveness of the GP model, and the conservatism introduced by uncertainty propagation.
- **Perfect tracking is rarely achievable in practice.** Both MPC and GP-MPC are subject to fundamental trade-offs between safety, robustness, performance, and computational complexity.
- **Further improvements may require richer data, better model structures, or hybrid learning/model-based approaches.**

# Reference 

[1] Hewing, L., Kabzan, J., & Zeilinger, M. N. (2020). Cautious Model Predictive Control Using Gaussian Process Regression. IEEE Transactions on Control Systems Technology, 28(6), 2736–2743.

[2] Wang, J., & Zhang, Y. (2024). A Tutorial on Gaussian Process Learning-based Model Predictive Control. arXiv preprint arXiv:2404.03689.

[3] A. Mesbah et al., "Fusion of Machine Learning and MPC under Uncertainty: What Advances Are on the Horizon?," 2022 American Control Conference (ACC), Atlanta, GA, USA, 2022, pp. 342-357.